## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sorong,ID,2022-04-02 16:39:33,-0.8833,131.2500,79.41,81,100,4.83,overcast clouds
1,1,Albany,US,2022-04-02 16:37:53,42.6001,-73.9662,46.17,43,4,4.00,clear sky
2,2,Avarua,CK,2022-04-02 16:39:33,-21.2078,-159.7750,73.45,94,85,2.30,overcast clouds
3,3,Cidreira,BR,2022-04-02 16:39:33,-30.1811,-50.2056,71.80,68,5,13.80,clear sky
4,4,Jamestown,US,2022-04-02 16:39:34,42.0970,-79.2353,42.03,48,0,4.61,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method. # 4a. Determine if there are any empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Sorong,ID,2022-04-02 16:39:33,-0.8833,131.2500,79.41,81,100,4.83,overcast clouds
1,1,Albany,US,2022-04-02 16:37:53,42.6001,-73.9662,46.17,43,4,4.00,clear sky
2,2,Avarua,CK,2022-04-02 16:39:33,-21.2078,-159.7750,73.45,94,85,2.30,overcast clouds
3,3,Cidreira,BR,2022-04-02 16:39:33,-30.1811,-50.2056,71.80,68,5,13.80,clear sky
4,4,Jamestown,US,2022-04-02 16:39:34,42.0970,-79.2353,42.03,48,0,4.61,clear sky
5,5,Vaini,TO,2022-04-02 16:39:34,-21.2000,-175.2000,71.76,100,20,0.00,few clouds
6,6,Punta Arenas,CL,2022-04-02 16:36:05,-53.1500,-70.9167,42.91,93,100,12.66,light intensity drizzle
7,7,Terekli-Mekteb,RU,2022-04-02 16:39:34,44.1671,45.8698,60.76,53,16,4.09,few clouds
8,8,Rikitea,PF,2022-04-02 16:39:35,-23.1203,-134.9692,74.17,57,48,8.19,scattered clouds
9,9,Ushuaia,AR,2022-04-02 16:39:35,-54.8000,-68.3000,42.42,93,100,6.91,drizzle


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count()

City_ID        589
City           589
Country        589
Date           589
Lat            589
Lng            589
Max Temp       589
Humidity       589
Cloudiness     589
Wind Speed     589
Description    589
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Sorong,ID,79.41,overcast clouds,-0.8833,131.2500,
1,Albany,US,46.17,clear sky,42.6001,-73.9662,
2,Avarua,CK,73.45,overcast clouds,-21.2078,-159.7750,
3,Cidreira,BR,71.80,clear sky,-30.1811,-50.2056,
4,Jamestown,US,42.03,clear sky,42.0970,-79.2353,
5,Vaini,TO,71.76,few clouds,-21.2000,-175.2000,
6,Punta Arenas,CL,42.91,light intensity drizzle,-53.1500,-70.9167,
7,Terekli-Mekteb,RU,60.76,few clouds,44.1671,45.8698,
8,Rikitea,PF,74.17,scattered clouds,-23.1203,-134.9692,
9,Ushuaia,AR,42.42,drizzle,-54.8000,-68.3000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    hotels
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# Double check rows
print(hotel_df.count())
print(clean_hotel_df.count())

City           589
Country        589
Max Temp       589
Description    589
Lat            589
Lng            589
Hotel Name     589
dtype: int64
City           589
Country        589
Max Temp       589
Description    589
Lat            589
Lng            589
Hotel Name     589
dtype: int64


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description/dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))